In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import GetOldTweets3 as got
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

## **Cleaning up csvs**

In our intial scraping, we had an issue with certain rural areas returning **the entire planet's tweets** so we had to remove those from our final dataset, as well as the following steps:
* Remove redundant headers
* Remove rows with no text or date

In [2]:
def combine_and_clean_shutdown_df(state_abbrev:str, cities_to_remove:list):
    '''
    This function combines and cleans dataframes after they were
    scraped from dirty, dirty Twitter. We need to go through a few
    similar steps based on our collection methods: remove broken
    cities, remove redundant headers, remove rows with no text or date
    
    -----------------------
    Parameters:
    
    'state_abbrev' : str, two-letter state abbreviation at the beginning
    of your auto-generated csv title
    
    'cities_to_remove' : list, A list of cities to remove from your dataset
    '''
    
    # Getting all of a state's filenames
    filenames = [filename for filename in os.listdir('./data/shutdown_data/') if filename.startswith(state_abbrev.upper())]
    # Create new df so the final df doesn't keep appending if you need to run again
    temp_df = pd.read_csv(f'./data/shutdown_data/{filenames[0]}')

    # Write a clean df
    df = pd.DataFrame(columns=temp_df.columns).to_csv(f'./data/shutdown_data/{state_abbrev.upper()}_full.csv', index=False)
    
    # Main cleaning loop
    for filename in filenames:
        df = pd.read_csv(f'./data/shutdown_data/{filename}')
        df.drop_duplicates(subset=['username', 'date'], keep='first', inplace=True) # need to remove any potential duplicates from overlapping city areas
        
        # Clean up final df
        df = df[(df['username'] != 'username') & # Removes headers leftover from scraping
               (~df['city'].isin(cities_to_remove))] # Removes the cities that we need to remove
        df.dropna(subset=['text', 'date'], inplace=True) # There were some nulls in the text and date column that are likely the result of deleted/private tweets
        df.to_csv(f'./data/shutdown_data/{state_abbrev.upper()}_full.csv', index=False, mode='a', header=False)

In [3]:
def combine_and_clean_reopening_df(state_abbrev:str, cities_to_remove:list):
    '''
    This function combines and cleans dataframes after they were
    scraped from dirty, dirty Twitter. We need to go through a few
    similar steps based on our collection methods: remove broken
    cities, remove redundant headers, remove rows with no text or date
    
    -----------------------
    Parameters:
    
    'state_abbrev' : str, two-letter state abbreviation at the beginning
    of your auto-generated csv title
    
    'cities_to_remove' : list, A list of cities to remove from your dataset
    '''
    
    # Getting all of a state's filenames
    filenames = [filename for filename in os.listdir('./data/reopening_data/') if filename.startswith(state_abbrev.upper())]
    # Create new df so the final df doesn't keep appending if you need to run again
    temp_df = pd.read_csv(f'./data/reopening_data/{filenames[0]}')

    # Write a clean df
    df = pd.DataFrame(columns=temp_df.columns).to_csv(f'./data/reopening_data/{state_abbrev.upper()}_full.csv', index=False)
    
    # Main cleaning loop
    for filename in filenames:
        df = pd.read_csv(f'./data/reopening_data/{filename}')
        df.drop_duplicates(subset=['username', 'date'], keep='first', inplace=True) # need to remove any potential duplicates from overlapping city areas
        
        # Clean up final df
        df = df[(df['username'] != 'username') & # Removes headers leftover from scraping
               (~df['city'].isin(cities_to_remove))] # Removes the cities that we need to remove
        df.dropna(subset=['text', 'date'], inplace=True) # There were some nulls in the text and date column that are likely the result of deleted/private tweets
        df.to_csv(f'./data/reopening_data/{state_abbrev.upper()}_full.csv', index=False, mode='a', header=False)

## **Cleaning Georgia Dataset**

In [6]:
ga_cities_to_remove = ['herndon', 'sunnyside', 'windsor', 'rowena', 'dublin']

In [27]:
combine_and_clean_shutdown_df('GA', ga_cities_to_remove)

In [63]:
# I accidentally grabbed too many days
df = pd.read_csv('./data/shutdown_data/GA_full.csv')
df['datetime'] = pd.to_datetime(df['date'], utc=True)

In [64]:
df['month_day'] = df['datetime'].dt.strftime('%m%d')

In [65]:
df = df[df['month_day'].astype(int) >= 330].reset_index(drop=True)

In [66]:
df.to_csv('./data/shutdown_data/GA_full.csv')

## **Cleaning Illinois Dataset**

In [57]:
il_cities_to_remove = ['ripley', 'island grove', 'pleasant grove', 'pickneyville']

In [58]:
combine_and_clean_shutdown_df('IL', il_cities_to_remove)

In [59]:
# I accidentally grabbed too many days
df = pd.read_csv('./data/shutdown_data/IL_full.csv')
df['datetime'] = pd.to_datetime(df['date'], utc=True)

C:\ProgramData\Anaconda3\envs\dsi\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
df['month_day'] = df['datetime'].dt.strftime('%m%d')

In [61]:
df = df[df['month_day'].astype(int) >= 317].reset_index(drop=True)

In [62]:
df.to_csv('./data/shutdown_data/IL_full.csv')

## **Cleaning Texas Dataset** -- going to do this on a separate notebook 

In [4]:
# tx_cities_to_remove = ['fort stolkhom']

In [5]:
# combine_and_clean_shutdown_df('TX', tx_cities_to_remove) 

In [7]:
# # I accidentally grabbed too many days
# df = pd.read_csv('./data/shutdown_data/IL_full.csv', low_memory=False)

In [8]:
# df['datetime'] = pd.to_datetime(df['date'], utc=True)

In [9]:
# df['month_day'] = df['datetime'].dt.strftime('%m%d')

In [10]:
# df = df[df['month_day'].astype(int) >= 317].reset_index(drop=True)

In [11]:
# df.to_csv('./data/shutdown_data/IL_full.csv')

# Once the reopening dates are scraped:
They should be added to this notebook following similar processes, but using the reopening function rather than the shutdown function